In [1]:
import csv
import os
import nltk
import pickle

In [2]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [3]:
file_name = "lyrics_1989_onwards.csv"

# Read the csv file
with open(file_name, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)


In [4]:
# convert ot pandas
import pandas as pd
df = pd.DataFrame(data)
df.columns = ['song', 'artist', 'lyrics']


In [5]:
# remove first row
df = df.iloc[1:]

In [6]:
df

,song,artist,lyrics
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...
...,...,...,...
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...
612,Got To Get,Leila K With Rob 'N' Raz,


In [7]:
# tokenize the lyrics
from nltk.tokenize import RegexpTokenizer

In [8]:
# tokenise
tokenizer = RegexpTokenizer(r'\w+')

df['tokens'] = df['lyrics'].apply(tokenizer.tokenize)

In [9]:
df

,song,artist,lyrics,tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold..."
...,...,...,...,...
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[Blue, Sky, Mine, Lyrics, Spoken, Intro, Music..."
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[Fly, High, Michelle, LyricsSix, o, clock, in,..."
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[Real, Love, LyricsDon, t, be, afraid, of, the..."
612,Got To Get,Leila K With Rob 'N' Raz,,[]


In [10]:
# remove numeric, stop words and short words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

stop_words = set(stopwords.words('english'))

# add pronouns to stop words

# list of pronouns

pronouns = ['i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

stop_words = stop_words.union(pronouns)

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def remove_stopwords(tokens):
    return [w for w in tokens if not w in stop_words]

def remove_punctuation(tokens):
    return [w for w in tokens if not w in string.punctuation]

def remove_numeric(tokens):
    return [w for w in tokens if not w.isnumeric()]

def remove_short_words(tokens):
    return [w for w in tokens if len(w) > 2]

def lemmatize(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]




In [11]:
df['tokens'] = df['tokens'].apply(remove_stopwords)
df['tokens'] = df['tokens'].apply(lemmatize)
df['tokens'] = df['tokens'].apply(remove_punctuation)
df['tokens'] = df['tokens'].apply(remove_numeric)
df['tokens'] = df['tokens'].apply(remove_short_words)

In [12]:
df

,song,artist,lyrics,tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, Paradise, Lyrics, Verse, She, c..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, Forget, Number, LyricsOkay, Okay, ..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[Rap, Collection, LyricsA, Hempstead, High, Lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, Rain, Lyrics, Verse, You, said, need, ..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, cold, hearted,..."
...,...,...,...,...
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[Blue, Sky, Mine, Lyrics, Spoken, Intro, Music..."
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[Fly, High, Michelle, LyricsSix, clock, mornin..."
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[Real, Love, LyricsDon, afraid, way, feel, rea..."
612,Got To Get,Leila K With Rob 'N' Raz,,[]


In [13]:
# remove any song with 0 tokens
df = df[df['tokens'].map(len) > 0]

In [14]:
df

,song,artist,lyrics,tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, Paradise, Lyrics, Verse, She, c..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, Forget, Number, LyricsOkay, Okay, ..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[Rap, Collection, LyricsA, Hempstead, High, Lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, Rain, Lyrics, Verse, You, said, need, ..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, cold, hearted,..."
...,...,...,...,...
608,All I Do Is Think Of You,Troop,All I Do Is Think of You Lyrics[Verse 1]\nI ca...,"[All, Think, You, Lyrics, Verse, wait, get, sc..."
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[Blue, Sky, Mine, Lyrics, Spoken, Intro, Music..."
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[Fly, High, Michelle, LyricsSix, clock, mornin..."
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[Real, Love, LyricsDon, afraid, way, feel, rea..."


In [15]:
# remove song name from tokens
song_tokens = df["song"]

# convert tokens to lowercase
df['tokens'] = df['tokens'].apply(lambda x: [item.lower() for item in x])

song_tokens = song_tokens.apply(tokenizer.tokenize)
song_tokens = song_tokens.apply(remove_stopwords)
song_tokens = song_tokens.apply(lemmatize)
song_tokens = song_tokens.apply(remove_punctuation)
song_tokens = song_tokens.apply(remove_numeric)
song_tokens = song_tokens.apply(remove_short_words)


/var/folders/gr/9n4v_bq578dgr67btw9kkmnm0000gn/T/ipykernel_8351/467598604.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].apply(lambda x: [item.lower() for item in x])


In [16]:
# add column to df
df['song_tokens'] = song_tokens

/var/folders/gr/9n4v_bq578dgr67btw9kkmnm0000gn/T/ipykernel_8351/3889431108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['song_tokens'] = song_tokens


In [17]:
df

,song,artist,lyrics,tokens,song_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[another, day, paradise, lyrics, verse, she, c...","[Another, Day, Paradise]"
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[baby, don, forget, number, lyricsokay, okay, ...","[Baby, Don, Forget, Number]"
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[rap, collection, lyricsa, hempstead, high, lo...","[Batdance, From, Batman]"
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[blame, rain, lyrics, verse, you, said, need, ...","[Blame, The, Rain]"
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[cold, hearted, lyrics, chorus, cold, hearted,...","[Cold, Hearted]"
...,...,...,...,...,...
608,All I Do Is Think Of You,Troop,All I Do Is Think of You Lyrics[Verse 1]\nI ca...,"[all, think, you, lyrics, verse, wait, get, sc...","[All, Think, You]"
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[blue, sky, mine, lyrics, spoken, intro, music...","[Blue, Sky, Mine]"
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[fly, high, michelle, lyricssix, clock, mornin...","[Fly, High, Michelle]"
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[real, love, lyricsdon, afraid, way, feel, rea...","[Real, Love]"


In [18]:
def remove_tokens(song_tokens, tokens):
    l = [w for w in tokens if w not in song_tokens]
    l = [w for w in l if w != "Lyrics"]
    l = [w for w in l if w != "Verse"]


    return l

In [19]:
df['tokens'] = df.apply(lambda x: remove_tokens(x['song_tokens'], x['tokens']), axis=1)

/var/folders/gr/9n4v_bq578dgr67btw9kkmnm0000gn/T/ipykernel_8351/1032254109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df.apply(lambda x: remove_tokens(x['song_tokens'], x['tokens']), axis=1)


In [20]:
df

,song,artist,lyrics,tokens,song_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[another, day, paradise, lyrics, verse, she, c...","[Another, Day, Paradise]"
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[baby, don, forget, number, lyricsokay, okay, ...","[Baby, Don, Forget, Number]"
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[rap, collection, lyricsa, hempstead, high, lo...","[Batdance, From, Batman]"
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[blame, rain, lyrics, verse, you, said, need, ...","[Blame, The, Rain]"
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[cold, hearted, lyrics, chorus, cold, hearted,...","[Cold, Hearted]"
...,...,...,...,...,...
608,All I Do Is Think Of You,Troop,All I Do Is Think of You Lyrics[Verse 1]\nI ca...,"[all, think, you, lyrics, verse, wait, get, sc...","[All, Think, You]"
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[blue, sky, mine, lyrics, spoken, intro, music...","[Blue, Sky, Mine]"
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[fly, high, michelle, lyricssix, clock, mornin...","[Fly, High, Michelle]"
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[real, love, lyricsdon, afraid, way, feel, rea...","[Real, Love]"


In [21]:
# remove profanities?

# https://www.freewebheaders.com/youtube-blacklist-words-free-and-youtube-comment-moderation/

with open('profanity.txt', 'r') as file:
    # split by comma
    profanities = file.read().split(',')
    profanities = [w.strip() for w in profanities]


In [22]:
profanities

['2 girls 1 cup',
 '2g1c',
 '4r5e',
 '5h1t',
 '5hit',
 'a$$',
 'a$$hole',
 'a_s_s',
 'a2m',
 'a54',
 'a55',
 'a55hole',
 'aeolus',
 'ahole',
 'alabama hot pocket',
 'alaskan pipeline',
 'anal',
 'anal impaler',
 'anal leakage',
 'analannie',
 'analprobe',
 'analsex',
 'anilingus',
 'anus',
 'apeshit',
 'ar5e',
 'areola',
 'areole',
 'arian',
 'arrse',
 'arse',
 'arsehole',
 'aryan',
 'ass',
 'ass fuck',
 'ass hole',
 'assault',
 'assbag',
 'assbagger',
 'assbandit',
 'assbang',
 'assbanged',
 'assbanger',
 'assbangs',
 'assbite',
 'assblaster',
 'assclown',
 'asscock',
 'asscracker',
 'asses',
 'assface',
 'assfaces',
 'assfuck',
 'assfucker',
 'ass-fucker',
 'assfukka',
 'assgoblin',
 'assh0le',
 'asshat',
 'ass-hat',
 'asshead',
 'assho1e',
 'asshole',
 'assholes',
 'asshopper',
 'asshore',
 'ass-jabber',
 'assjacker',
 'assjockey',
 'asskiss',
 'asskisser',
 'assklown',
 'asslick',
 'asslicker',
 'asslover',
 'assman',
 'assmaster',
 'assmonkey',
 'assmucus',
 'assmunch',
 'assmunch

In [23]:
df['tokens'] = df['tokens'].apply(lambda x: [w for w in x if w not in profanities])

/var/folders/gr/9n4v_bq578dgr67btw9kkmnm0000gn/T/ipykernel_8351/1730082223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].apply(lambda x: [w for w in x if w not in profanities])


In [24]:
df

,song,artist,lyrics,tokens,song_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[another, day, paradise, lyrics, verse, she, c...","[Another, Day, Paradise]"
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[baby, don, forget, number, lyricsokay, okay, ...","[Baby, Don, Forget, Number]"
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[rap, collection, lyricsa, hempstead, high, lo...","[Batdance, From, Batman]"
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[blame, rain, lyrics, verse, you, said, need, ...","[Blame, The, Rain]"
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[cold, hearted, lyrics, chorus, cold, hearted,...","[Cold, Hearted]"
...,...,...,...,...,...
608,All I Do Is Think Of You,Troop,All I Do Is Think of You Lyrics[Verse 1]\nI ca...,"[all, think, you, lyrics, verse, wait, get, sc...","[All, Think, You]"
609,Blue Sky Mine,Midnight Oil,Blue Sky Mine Lyrics[Spoken Intro (Music video...,"[blue, sky, mine, lyrics, spoken, intro, music...","[Blue, Sky, Mine]"
610,Fly High Michelle,Enuff Z'Nuff,Fly High Michelle LyricsSix o'clock in the mor...,"[fly, high, michelle, lyricssix, clock, mornin...","[Fly, High, Michelle]"
611,Real Love,Skyy,Real Love LyricsDon't be afraid of the way you...,"[real, love, lyricsdon, afraid, way, feel, rea...","[Real, Love]"


In [25]:
# create list of all tokens 
all_tokens = df['tokens'].tolist()

In [26]:
from gensim.corpora import Dictionary
dictionary = Dictionary(all_tokens)
dictionary.filter_extremes(no_below = 20, no_above = 0.8)

In [27]:
from gensim.corpora import MmCorpus
gensim_corpus = [dictionary.doc2bow(song) for song in all_tokens]
temp = dictionary[0]
id2word = dictionary.id2token

In [28]:
chunksize = 2000
passes = 20
iterations = 400
num_topics = 6

In [29]:
from gensim.models import LdaModel
lda_model = LdaModel(
corpus=gensim_corpus,
id2word=id2word,
chunksize=chunksize,
alpha='auto',
eta='auto',
iterations=iterations,
num_topics=num_topics,
passes=passes
)


In [30]:
from gensim.models.coherencemodel import CoherenceModel
coherencemodel = CoherenceModel(model=lda_model, texts=all_tokens, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.43846748129793495


In [31]:
!pip install pyLDAvis

In [32]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
#pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data, './Lyrics_LDA_k_'+ str(num_topics) +'.html')

/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anac